In [16]:
import sys
import os
import pickle

import importlib
import clustering_utils
importlib.reload(clustering_utils)
import utils
importlib.reload(utils)
sys.path.append('../chat-intents/chatintents')
import chatintents
importlib.reload(chatintents)

import numpy as np
import pandas as pd
from hyperopt import hp
from sentence_transformers import SentenceTransformer
import json
import argparse
import yaml

import plotly.express as px
import matplotlib.pyplot as plt
import umap
import fast_hdbscan
import time
import torch
import re
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from clustering_utils import remove_duplicates, get_bert_topics, get_chat_intents_topics, get_clustering_metrics, get_most_common_company, save_input_with_params, get_clustering_metrics, plot_best_clusters, save_embeddings, perform_clustering, remove_stopwords
from clustering_utils import is_supervised_training, split_supervised_training_df
from utils import load_model, create_replace_no_tags_embeddings
from chatintents import ChatIntents

from nltk.corpus import stopwords
import nltk
from bertopic.representation import MaximalMarginalRelevance
import openai
from bertopic.representation import OpenAI
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bartekjezierski/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bartekjezierski/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
params = {}
params['DROP_DUPLICATES'] = True
params['TOPIC_MODEL'] = "bert" # "chat_intents"
params['INPUT_DIR'] = '../glanos-data/clustering/input/'
params['OUTPUT_DIR'] = '../glanos-data/clustering/output/'
params['BAYESIAN_SEARCH_METRIC'] = 'topic_diversity+bert' # 'outliers', 'topic_overlap+bert', 'topic_diversity+bert'
params['REMOVE_STOPWORDS'] = True
params['KEEP_ONLY_BAD_LABELS'] = True
params['INFERENCE_LABELS'] = ['OTHER|ENTITY', 'OTHER', 'ENTITY', 'ENTITY|OTHER']
params['SEMI_SUPERVISED'] = True
params['PENALTY'] = 'label_count' # ['label_count', 'outside_range']
params['LABEL_RANGE'] = (28, 50) # applicable only if outside_range in PENALTY
params['OUTSIDE_RANGE_PENALTY'] = 10 # applicable only if outside_range in PENALTY
params['TOP_K_TOPICS'] = 10
params['N_NEIGHBORS_RANGE'] = range(50, 80)
params['N_COMPONENTS_RANGE'] = range(4, 15)
params['MIN_CLUSTER_SIZE_RANGE'] = range(25, 60)
params['MIN_SAMPLE_SIZE_RANGE'] = range(1, 4)
params['MAX_EVALS'] = 15 # best: 20, only relevant for the hyperparameter search


In [18]:
def run_bayesian_search(model, params, df=None, param_output_file_path=None):
    hspace = {
        "n_neighbors": hp.choice('n_neighbors', params['N_NEIGHBORS_RANGE']),
        "n_components": hp.choice('n_components', params['N_COMPONENTS_RANGE']),
        "min_cluster_size": hp.choice('min_cluster_size', params['MIN_CLUSTER_SIZE_RANGE']),
        "min_samples": hp.choice('min_samples', params['MIN_SAMPLE_SIZE_RANGE']), # the higher it is, the more strict will clustering be and have more outliers
        "random_state": 42
    }

    label_lower = params['LABEL_RANGE'][0]
    label_upper = params['LABEL_RANGE'][1]
    best_params, best_clusters, trials = model.bayesian_search(space=hspace,
                          label_lower=label_lower,
                          label_upper=label_upper,
                          params=params,
                          df=df
                          )
    if param_output_file_path:
        save_input_with_params(best_params, df, param_output_file_path)
    return best_params

In [19]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation

def cluster(input_file_name, params, hf_model_name="", topic_model=params['TOPIC_MODEL']):
    params['HF_MODEL_NAME'] = hf_model_name
    input_file_path = params['INPUT_DIR'] + input_file_name
    print('Input file', input_file_path)
    
    if "data2" in input_file_path and "replace_no_tags" in input_file_path and 'seb' in input_file_path:
        dataset_file_path = '../glanos-data/clustering/big_consulting_export_replace_seb.tsv' 
    elif "data2" in input_file_path and "replace_no_tags" in input_file_path:
        dataset_file_path = '../glanos-data/clustering/big_consulting_export_replace_no_duplicates.tsv' 
    elif "data2" in input_file_path:
        dataset_file_path = '../glanos-data/clustering/big_consulting_export_data2_no_duplicates.tsv' 
    else:
        dataset_file_path = '../glanos-data/clustering/big_consulting_export.tsv'
        
    if not os.path.exists(input_file_path):
        if os.path.exists(input_file_path.replace('_params', '')):
            input_file_path = input_file_path.replace('_params', '')
        else:
            save_embeddings(hf_model_name, 
                        dataset_file_path, 
                        input_file_path, 
                        "snippet" if not "replace" in input_file_path else ("replace_no_tags" if "replace_no_tags" in input_file_path else "replace"),
                        params)

    with open(input_file_path, 'r') as f:
        data = json.load(f)

    if 'items' in data:
        df = pd.json_normalize(data['items'])
        df = df.dropna()
        if params['DROP_DUPLICATES']:
            df = remove_duplicates(df)

        params['SNIPPET_COLUMN_NAME'] = "snippet" if not "replace_no_tags" in input_file_path else "replace_no_tags"
        snippets = df[params['SNIPPET_COLUMN_NAME']].tolist()
        
        model_name = globals()["model"] if "model" in globals() else "sbert"
        if 'embedding' in df.columns:
            embeddings = df['embedding'].tolist()
        else:
            print('Embeddings snippets like ', snippets[0])
            if model_name == "sbert":
                embeddings = embed_sbert(snippets)
            elif model_name == "glove":
                embeddings = encode_glove(snippets)
            else:
                return "Invalid data format.", 400
            df['embedding'] = embeddings.tolist()

        if is_supervised_training(df, params):
            print('Supervised')
            training_df, inference_df = split_supervised_training_df(df, params)
            training_embeddings = training_df["embedding"].to_list()
            training_titles = training_df["snippet"].to_list()
            inference_embeddings = inference_df["embedding"].to_list()
            inference_titles = inference_df["snippet"].to_list()

            model = ChatIntents(training_embeddings, inference_embeddings, training_df, inference_df, model_name)
        else:
            model = ChatIntents(embeddings, embeddings, df, df, model_name)
            inference_df = df

        if 'parameters' in data:
            hdbscan_umap_params = data['parameters']
#             hdbscan_umap_params = {"min_cluster_size": 38, "min_samples": 3, "n_components": 5, "n_neighbors": 51, "outside_range_penalty": 2, "random_state": 42}
            if topic_model == "chat_intents":
                hdbscan_clustering = perform_clustering(model, 
                                                    hdbscan_umap_params['n_neighbors'], 
                                                    hdbscan_umap_params['n_components'], 
                                                    hdbscan_umap_params['min_cluster_size'],
                                                    hdbscan_umap_params['min_samples'] if 'min_samples' in hdbscan_umap_params else 1
                                                )
        else:
            hdbscan_umap_params = run_bayesian_search(model, 
                                params,
                                df=df, 
                                param_output_file_path=input_file_path.replace("_params", "").replace(".json", "_params.json")
                               )
            print()
            hdbscan_clustering = model.best_clusters
        
        params.update(hdbscan_umap_params)

        start_time_topic_modelling = time.time()
        if topic_model == "chat_intents":
            print("Topic modelling method: ChatIntents")
            cluster_topics = get_chat_intents_topics(model, inference_df)
            probabilities = hdbscan_clustering.probabilities_
            cluster_labels = hdbscan_clustering.labels_
#         elif topic_model == "bert":
        else:
            print("Topic modelling method: BERTopic")
            cluster_topics, inference_df, topic_model = get_bert_topics(df, params)
            probabilities = inference_df['Probability'].to_numpy()
            cluster_labels = inference_df['cluster_id'].to_numpy()

        print(f"Elapsed total time for topic modelling: {time.time() - start_time_topic_modelling} seconds")
        if len(inference_df) != len(cluster_labels):
            print(f"Cluster labels not collected correctly ({len(inference_df)} vs. {len(cluster_labels)})")

        label_count, cost = model.score_clusters(probabilities, cluster_labels)
        print(f"{model_name} results")
        print(f"Label count: {label_count}")
        print(f"Loss: {cost}")
        
            
        if "SHOW" in params and params["SHOW"]:
            if is_supervised_training(inference_df, params):
                print("is supervised")
                umap_model_supervised = umap.UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit(training_embeddings)
                reduced_embeddings_supervised = umap_model_supervised.transform(inference_embeddings)
                fig = topic_model.visualize_documents(inference_titles, reduced_embeddings=reduced_embeddings_supervised, custom_labels=True, hide_annotations=True)
                fig.show()  
            else:
                df = plot_best_clusters(model, inference_df, cluster_labels, params, filename=input_file_name.replace('.json', ''))
#         df = plot_best_clusters(model, inference_df, hdbscan_clustering.labels_, params, filename=input_file_name.replace('.json', ''))
#         df['probability'] = hdbscan_clustering.probabilities_
#         label_count = df['cluster_id'].nunique()

        params.update({'label_count': label_count, 'cost': cost})

        print("dataset_file_path", dataset_file_path)
        get_clustering_metrics(inference_df, cluster_topics, params, input_file=dataset_file_path)

        return model, inference_df, cluster_topics, params, topic_model
    else:
        return jsonify({"error_msg": "Invalid data format."})


globals()["model"] = "sbert"

In [20]:
def print_results(input_file):
    if 'sbert_output_df' in globals():
        print("Baseline SBERT:")
        sbert_overlaps = get_clustering_metrics(sbert_output_df, sbert_cluster_topics, sbert_params, input_file=input_file)

    if 'baai_output_df' in globals():
        print("Baseline BAAI:")
        baai_overlaps = get_clustering_metrics(baai_output_df, baai_cluster_topics, baai_params, input_file=input_file)


    if 'similarity_baai_output_df' in globals():
        print("Similarity (SBERT):")
        similarity_baai_overlaps = get_clustering_metrics(similarity_baai_output_df, similarity_baai_cluster_topics, similarity_baai_params, input_file=input_file)

    if 'similarity_sbert_output_df' in globals():
        print("Similarity (BAAI):")
        similarity_sbert_overlaps = get_clustering_metrics(similarity_sbert_output_df, similarity_sbert_cluster_topics, similarity_sbert_params, input_file=input_file)

    print("Below only SBERT:")

    if 'similarity_ai_car_sbert_output_df' in globals():
        print("Similarity trained on ai+car dataset:")
        similarity_ai_car_sbert_overlaps = get_clustering_metrics(similarity_ai_car_sbert_output_df, similarity_ai_car_sbert_cluster_topics, similarity_ai_car_sbert_params, input_file=input_file)

    if 'similarity_distill_consistency_output_df' in globals():
        print("Similarity distill-consistency model trained on consulting dataset:")
        similarity_distill_consistency_overlaps = get_clustering_metrics(similarity_distill_consistency_output_df, similarity_distill_consistency_cluster_topics, similarity_distill_consistency_params, input_file=input_file)

    if 'similarity_ai_car_distill_consistency_output_df' in globals():
        print("Similarity distill-consistency model trained on ai+car dataset:")
        similarity_ai_car_distill_consistency_overlaps = get_clustering_metrics(similarity_ai_car_distill_consistency_output_df, similarity_ai_car_distill_consistency_cluster_topics, similarity_ai_car_distill_consistency_params, input_file=input_file)


    if 'classification_ai_car_sbert_output_df' in globals():
        print("Classification on ai+car:")
        classification_ai_car_sbert_overlaps = get_clustering_metrics(classification_ai_car_sbert_output_df, classification_ai_car_sbert_cluster_topics, classification_ai_car_sbert_params, input_file=input_file)

    if 'classification_output_df' in globals():
        print("Classification on consulting:")
        classification_overlaps = get_clustering_metrics(classification_output_df, classification_cluster_topics, classification_params, input_file=input_file)


    print("Combined training")

    if 'combined_ai_car_class_consulting_sim_output_df' in globals():
        print("First classification on ai+car dataset, then similarity on consulting dataset:")
        combined_ai_car_class_consulting_sim_overlaps = get_clustering_metrics(combined_ai_car_class_consulting_sim_output_df, combined_ai_car_class_consulting_sim_cluster_topics, combined_ai_car_class_consulting_sim_params, input_file=input_file)

    if 'combined_consulting_sim_ai_car_class_output_df' in globals():
        print("First similarity on consulting dataset, then classification on ai+car dataset:")
        combined_consulting_sim_ai_car_class_overlaps = get_clustering_metrics(combined_consulting_sim_ai_car_class_output_df, combined_consulting_sim_ai_car_class_cluster_topics, combined_consulting_sim_ai_car_class_params, input_file=input_file)

    if 'combined_consulting_sim_and_ai_car_class_output_df' in globals():
        print("Together similarity on consulting dataset, and classification on ai+car dataset:")
        combined_consulting_sim_and_ai_car_class_overlaps = get_clustering_metrics(combined_consulting_sim_and_ai_car_class_output_df, combined_consulting_sim_and_ai_car_class_cluster_topics, combined_consulting_sim_and_ai_car_class_params, input_file=input_file)

    if 'combined_ai_car_sim_and_class_output_df' in globals():
        print("Together similarity and classification on ai+car dataset:")
        combined_ai_car_sim_and_class_overlaps = get_clustering_metrics(combined_ai_car_sim_and_class_output_df, combined_ai_car_sim_and_class_cluster_topics, combined_ai_car_sim_and_class_params, input_file=input_file)

    if 'combined_ai_car_sim_class_output_df' in globals():
        print("First similarity, then classification on ai+car dataset:")
        combined_ai_car_sim_class_overlaps = get_clustering_metrics(combined_ai_car_sim_class_output_df, combined_ai_car_sim_class_cluster_topics, combined_ai_car_sim_class_params, input_file=input_file)

    if 'combined_ai_car_class_sim_output_df' in globals():
        print("First classification, then similarity on ai+car dataset:")
        combined_ai_car_class_sim_overlaps = get_clustering_metrics(combined_ai_car_class_sim_output_df, combined_ai_car_class_sim_cluster_topics, combined_ai_car_class_sim_params, input_file=input_file)

    if 'combined_consulting_output_df' in globals():
        print("Together classification and similarity on consulting dataset:")
        combined_consulting_overlaps = get_clustering_metrics(combined_consulting_output_df, combined_consulting_cluster_topics, combined_consulting_params, input_file=input_file)
        
    if 'classification_ai_car_data_aug_output_df' in globals():
        print("Classification on augmented dataset:")
        classification_ai_car_data_aug_overlaps = get_clustering_metrics(classification_ai_car_data_aug_output_df, classification_ai_car_data_aug_cluster_topics, classification_ai_car_data_aug_params, input_file=input_file)
        

# Experiment 1

In [24]:
# on replace_no_tags dataset
# diversity loss

data2_input_file = '../glanos-data/clustering/big_consulting_export_replace.tsv'
params['BAYESIAN_SEARCH_METRIC'] = 'topic_diversity+bert'
params["SHOW"] = False

# sbert_model, sbert_output_df, sbert_cluster_topics, sbert_params, sbert_topic_model = cluster('baseline_data2_semi_replace_no_tags_td_params.json', params, hf_model_name='sentence-transformers/all-MiniLM-L12-v2')
# baai_model, baai_output_df, baai_cluster_topics, baai_params, baai_topic_model = cluster('baai_baseline_data2_semi_replace_no_tags_td_params.json', params, hf_model_name='BAAI/bge-base-en-v1.5')

# similarity_sbert_model, similarity_sbert_output_df, similarity_sbert_cluster_topics, similarity_sbert_params, similarity_sbert_topic_model = cluster('similarity_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-similarity")
# similarity_baai_model, similarity_baai_output_df, similarity_baai_cluster_topics, similarity_baai_params, similarity_baai_topic_model = cluster('best_similarity_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-similarity-best-baai")
# similarity_ai_car_sbert_model, similarity_ai_car_sbert_output_df, similarity_ai_car_sbert_cluster_topics, similarity_ai_car_sbert_params, similarity_ai_car_sbert_topic_model = cluster('similarity_ai_car_data2_semi_replace_no_tags_td_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-ai-car')
# similarity_distill_consistency_model, similarity_distill_consistency_output_df, similarity_distill_consistency_cluster_topics, similarity_distill_consistency_params, similarity_distill_consistency_topic_model = cluster('similarity_distill_consistency_data2_semi_replace_no_tags_td_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-distill-consistency')
# similarity_ai_car_distill_consistency_model, similarity_ai_car_distill_consistency_output_df, similarity_ai_car_distill_consistency_cluster_topics, similarity_ai_car_distill_consistency_params, similarity_ai_car_distill_consistency_topic_model = cluster('similarity_ai_car_distill_consistency_data2_semi_replace_no_tags_td_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-ai_car-distill-consistency')


# classification_model, classification_output_df, classification_cluster_topics, classification_params, classification_topic_model = cluster('classification_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification")
# classification_ai_car_sbert_model, classification_ai_car_sbert_output_df, classification_ai_car_sbert_cluster_topics, classification_ai_car_sbert_params, classification_ai_car_sbert_topic_model = cluster('classification_ai_car_sbert_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification-ai_car-sbert")
# combined_ai_car_class_consulting_sim_model, combined_ai_car_class_consulting_sim_output_df, combined_ai_car_class_consulting_sim_cluster_topics, combined_ai_car_class_consulting_sim_params, combined_ai_car_class_consulting_sim_topic_model = cluster('combined_ai_car_class_consulting_sim_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-class-consulting-sim")
# combined_consulting_sim_ai_car_class_model, combined_consulting_sim_ai_car_class_output_df, combined_consulting_sim_ai_car_class_cluster_topics, combined_consulting_sim_ai_car_class_params, combined_consulting_sim_ai_car_class_topic_model = cluster('combined_consulting_sim_ai_car_class_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-consulting-sim-ai_car-class")
# combined_consulting_sim_and_ai_car_class_model, combined_consulting_sim_and_ai_car_class_output_df, combined_consulting_sim_and_ai_car_class_cluster_topics, combined_consulting_sim_and_ai_car_class_params, combined_consulting_sim_and_ai_car_class_topic_model = cluster('combined_consulting_sim_and_ai_car_class_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-consulting-sim-and-ai_car-class")
# combined_ai_car_sim_and_class_model, combined_ai_car_sim_and_class_output_df, combined_ai_car_sim_and_class_cluster_topics, combined_ai_car_sim_and_class_params, combined_ai_car_sim_and_class_topic_model = cluster('combined_ai_car_sim_and_class_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-sim-and-class")
# combined_ai_car_sim_class_model, combined_ai_car_sim_class_output_df, combined_ai_car_sim_class_cluster_topics, combined_ai_car_sim_class_params, combined_ai_car_sim_class_topic_model = cluster('combined_ai_car_sim_class_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-sim-class")
# combined_ai_car_class_sim_model, combined_ai_car_class_sim_output_df, combined_ai_car_class_sim_cluster_topics, combined_ai_car_class_sim_params, combined_ai_car_class_sim_topic_model = cluster('combined_ai_car_class_sim_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-class-sim")
# combined_consulting_model, combined_consulting_output_df, combined_consulting_cluster_topics, combined_consulting_params, combined_consulting_topic_model = cluster('combined_consulting_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/combined-consulting")

ex1_classification_ai_car_data_aug_model, ex1_classification_ai_car_data_aug_output_df, ex1_classification_ai_car_data_aug_cluster_topics, ex1_classification_ai_car_data_aug_params, ex1_classification_ai_car_data_aug_topic_model = cluster('classification_ai_car_data_aug_data2_semi_replace_no_tags_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification-ai_car-data_aug")



Input file ../glanos-data/clustering/input/classification_ai_car_data_aug_data2_semi_replace_no_tags_td_params.json
Supervised
Topic modelling method: BERTopic
Fitting the BERT model on 1278 snippets


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

2023-10-27 18:25:34,792 - BERTopic - Transformed documents to Embeddings
2023-10-27 18:25:37,268 - BERTopic - Reduced dimensionality
2023-10-27 18:25:37,283 - BERTopic - Clustered reduced embeddings


BERT model labelling 3940 snippets
inference_snippets 3940


Batches:   0%|          | 0/124 [00:00<?, ?it/s]

2023-10-27 18:26:13,425 - BERTopic - Reduced dimensionality
2023-10-27 18:26:13,475 - BERTopic - Predicted clusters


Elapsed total time for topic modelling: 55.39846897125244 seconds
sbert results
Label count: 8
Loss: 0.03807106598984772
dataset_file_path ../glanos-data/clustering/big_consulting_export_replace_no_duplicates.tsv
Overlap count of length 1: 1
Overlap count of length 2: 2
Overlap count of length 3: 0
Overlap count of length 4 or more: 0
Overlap loss:  0.28
Diversity loss:  1.06
Number of clusters: 8
Avg cluster size: 437.77777777777777
Median cluster size: 372.0 , standard deviation: 477.9167239820431
Median top company occurence: 44.776119402985074
Cost (outliers): 3.81%



In [27]:
ex1_classification_ai_car_data_aug_cluster_topics, ex3_classification_ai_car_data_aug_cluster_topics

([['fined',
   'state',
   'role',
   'issued',
   'consultancy',
   'linked',
   'welcomes opportunity',
   'concerns',
   'denying',
   'didn raise'],
  ['ai',
   'cloud',
   'announced',
   'technology',
   'partnership',
   'platform',
   'services',
   'new',
   'business',
   'digital'],
  ['results',
   'reported',
   'revenue',
   'net',
   'quarter',
   'fiscal',
   'year',
   'growth',
   'investment',
   'share'],
  ['served',
   'prior',
   'joining',
   'prior joining',
   'partner',
   'years',
   'practice',
   'roles',
   'spent',
   'audit'],
  ['acquisition',
   'acquire',
   'sustainability',
   'announced',
   'services',
   'domus',
   'green domus',
   'sustainability consultancy',
   'green',
   'company'],
  ['tax',
   'law',
   'laws',
   'trade',
   'tax laws',
   'pay',
   'scandal',
   'agreed',
   'information',
   'confidential'],
  ['tax',
   'government',
   'partners',
   'confidential',
   'treasury',
   'leave',
   'internal',
   'confidential governm

In [7]:
data2_input_file = '../glanos-data/clustering/big_consulting_export_replace_no_duplicates.tsv'
print_results(input_file=data2_input_file)


Baseline SBERT:
Overlap count of length 1: 11
Overlap count of length 2: 4
Overlap count of length 3: 2
Overlap count of length 4 or more: 0
Overlap loss:  1.84
Diversity loss:  1.15
Number of clusters: 8
Median cluster size: 137.0 , standard deviation: 392.4709189509743
Median top company occurence: 46.875
Cost (outliers): 3.81%

Baseline BAAI:
Overlap count of length 1: 9
Overlap count of length 2: 2
Overlap count of length 3: 1
Overlap count of length 4 or more: 1
Overlap loss:  2.03
Diversity loss:  1.15
Number of clusters: 8
Median cluster size: 142.0 , standard deviation: 499.85835863594895
Median top company occurence: 31.164209456024405
Cost (outliers): 3.81%

Similarity (SBERT):
Overlap count of length 1: 9
Overlap count of length 2: 4
Overlap count of length 3: 1
Overlap count of length 4 or more: 1
Overlap loss:  2.28
Diversity loss:  1.18
Number of clusters: 8
Median cluster size: 145.0 , standard deviation: 397.3414461795686
Median top company occurence: 28.205128205128204

# Experiment 2

In [6]:
# on replace_no_tags dataset
# overlap loss

data2_input_file = '../glanos-data/clustering/big_consulting_export_replace.tsv'
params['BAYESIAN_SEARCH_METRIC'] = 'topic_overlap+bert'

sbert_model, sbert_output_df, sbert_cluster_topics, sbert_params, sbert_topic_model = cluster('baseline_data2_semi_replace_no_tags_to_params.json', params, hf_model_name='sentence-transformers/all-MiniLM-L12-v2')
baai_model, baai_output_df, baai_cluster_topics, baai_params, baai_topic_model = cluster('baai_baseline_data2_semi_replace_no_tags_to_params.json', params, hf_model_name='BAAI/bge-base-en-v1.5')

similarity_sbert_model, similarity_sbert_output_df, similarity_sbert_cluster_topics, similarity_sbert_params, similarity_sbert_topic_model = cluster('similarity_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-similarity")
similarity_baai_model, similarity_baai_output_df, similarity_baai_cluster_topics, similarity_baai_params, similarity_baai_topic_model = cluster('best_similarity_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-similarity-best-baai")
similarity_ai_car_sbert_model, similarity_ai_car_sbert_output_df, similarity_ai_car_sbert_cluster_topics, similarity_ai_car_sbert_params, similarity_ai_car_sbert_topic_model = cluster('similarity_ai_car_data2_semi_replace_no_tags_to_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-ai-car')
similarity_distill_consistency_model, similarity_distill_consistency_output_df, similarity_distill_consistency_cluster_topics, similarity_distill_consistency_params, similarity_distill_consistency_topic_model = cluster('similarity_distill_consistency_data2_semi_replace_no_tags_to_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-distill-consistency')
similarity_ai_car_distill_consistency_model, similarity_ai_car_distill_consistency_output_df, similarity_ai_car_distill_consistency_cluster_topics, similarity_ai_car_distill_consistency_params, similarity_ai_car_distill_consistency_topic_model = cluster('similarity_ai_car_distill_consistency_data2_semi_replace_no_tags_to_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-ai_car-distill-consistency')


classification_model, classification_output_df, classification_cluster_topics, classification_params, classification_topic_model = cluster('classification_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification")
classification_ai_car_sbert_model, classification_ai_car_sbert_output_df, classification_ai_car_sbert_cluster_topics, classification_ai_car_sbert_params, classification_ai_car_sbert_topic_model = cluster('classification_ai_car_sbert_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification-ai_car-sbert")
combined_ai_car_class_consulting_sim_model, combined_ai_car_class_consulting_sim_output_df, combined_ai_car_class_consulting_sim_cluster_topics, combined_ai_car_class_consulting_sim_params, combined_ai_car_class_consulting_sim_topic_model = cluster('combined_ai_car_class_consulting_sim_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-class-consulting-sim")
combined_consulting_sim_ai_car_class_model, combined_consulting_sim_ai_car_class_output_df, combined_consulting_sim_ai_car_class_cluster_topics, combined_consulting_sim_ai_car_class_params, combined_consulting_sim_ai_car_class_topic_model = cluster('combined_consulting_sim_ai_car_class_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-consulting-sim-ai_car-class")
combined_consulting_sim_and_ai_car_class_model, combined_consulting_sim_and_ai_car_class_output_df, combined_consulting_sim_and_ai_car_class_cluster_topics, combined_consulting_sim_and_ai_car_class_params, combined_consulting_sim_and_ai_car_class_topic_model = cluster('combined_consulting_sim_and_ai_car_class_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-consulting-sim-and-ai_car-class")
combined_ai_car_sim_and_class_model, combined_ai_car_sim_and_class_output_df, combined_ai_car_sim_and_class_cluster_topics, combined_ai_car_sim_and_class_params, combined_ai_car_sim_and_class_topic_model = cluster('combined_ai_car_sim_and_class_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-sim-and-class")
combined_ai_car_sim_class_model, combined_ai_car_sim_class_output_df, combined_ai_car_sim_class_cluster_topics, combined_ai_car_sim_class_params, combined_ai_car_sim_class_topic_model = cluster('combined_ai_car_sim_class_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-sim-class")
combined_ai_car_class_sim_model, combined_ai_car_class_sim_output_df, combined_ai_car_class_sim_cluster_topics, combined_ai_car_class_sim_params, combined_ai_car_class_sim_topic_model = cluster('combined_ai_car_class_sim_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-class-sim")
combined_consulting_model, combined_consulting_output_df, combined_consulting_cluster_topics, combined_consulting_params, combined_consulting_topic_model = cluster('combined_consulting_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/combined-consulting")
classification_ai_car_data_aug_model, classification_ai_car_data_aug_output_df, classification_ai_car_data_aug_cluster_topics, classification_ai_car_data_aug_params, classification_ai_car_data_aug_topic_model = cluster('classification_ai_car_data_aug_data2_semi_replace_no_tags_to_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification-ai_car-data_aug")



Input file ../glanos-data/clustering/input/baseline_data2_semi_replace_no_tags_to_params.json
Supervised
Topic modelling method: BERTopic
Fitting the BERT model on 1278 snippets


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

2023-10-26 19:21:45,668 - BERTopic - Transformed documents to Embeddings
2023-10-26 19:21:50,092 - BERTopic - Reduced dimensionality
2023-10-26 19:21:50,116 - BERTopic - Clustered reduced embeddings


BERT model labelling 3940 snippets
inference_snippets 3940


Batches:   0%|          | 0/124 [00:00<?, ?it/s]

...

BERT model labelling 3940 snippets                                             
inference_snippets                                                             
3940                                                                           
  0%|                                   | 0/15 [00:20<?, ?trial/s, best loss=?]

Batches:   0%|          | 0/124 [00:00<?, ?it/s]

2023-10-26 19:40:54,989 - BERTopic - Reduced dimensionality
2023-10-26 19:40:55,035 - BERTopic - Predicted clusters
job exception: 'cluster_id'



  0%|                                   | 0/15 [00:56<?, ?trial/s, best loss=?]


KeyError: 'cluster_id'

In [12]:
# TODO run
params['BAYESIAN_SEARCH_METRIC'] = 'topic_overlap+bert'



Input file ../glanos-data/clustering/input/classification_ai_car_data_aug_data2_semi_replace_no_tags_to_params.json
Supervised
Fitting the BERT model on 1278 snippets                                        
  0%|                                   | 0/15 [00:11<?, ?trial/s, best loss=?]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

2023-10-26 21:04:35,163 - BERTopic - Transformed documents to Embeddings
2023-10-26 21:04:39,002 - BERTopic - Reduced dimensionality
2023-10-26 21:04:39,021 - BERTopic - Clustered reduced embeddings


BERT model labelling 3940 snippets                                             
inference_snippets                                                             
3940                                                                           
  0%|                                   | 0/15 [00:22<?, ?trial/s, best loss=?]

Batches:   0%|          | 0/124 [00:00<?, ?it/s]

...

2023-10-26 21:20:41,219 - BERTopic - Transformed documents to Embeddings
2023-10-26 21:20:44,623 - BERTopic - Reduced dimensionality
2023-10-26 21:20:44,636 - BERTopic - Clustered reduced embeddings


BERT model labelling 3940 snippets
inference_snippets 3940


Batches:   0%|          | 0/124 [00:00<?, ?it/s]

2023-10-26 21:21:14,640 - BERTopic - Reduced dimensionality
2023-10-26 21:21:14,687 - BERTopic - Predicted clusters


Elapsed total time for topic modelling: 50.21725106239319 seconds
sbert results
Label count: 6
Loss: 0.003045685279187817
dataset_file_path ../glanos-data/clustering/big_consulting_export_replace_no_duplicates.tsv
Overlap count of length 1: 0
Overlap count of length 2: 1
Overlap count of length 3: 0
Overlap count of length 4 or more: 0
Overlap loss:  0.17
Diversity loss:  1.03
Number of clusters: 6
Median cluster size: 586.0 , standard deviation: 554.6841944345949
Median top company occurence: 33.666666666666664
Cost (outliers): 0.30%



In [ ]:
data2_input_file = '../glanos-data/clustering/big_consulting_export_replace_no_duplicates.tsv'
print_results(input_file=data2_input_file)


# Experiment 3

In [23]:
# on 'snippet' column
params['BAYESIAN_SEARCH_METRIC'] = 'topic_diversity+bert'
params['MAX_EVALS'] = 15
params["SHOW"] = False

# sbert_model, sbert_output_df, sbert_cluster_topics, sbert_params, sbert_topic_model = cluster('baseline_data2_semi_td_temp_params.json', params, hf_model_name='sentence-transformers/all-MiniLM-L12-v2')
# baai_model, baai_output_df, baai_cluster_topics, baai_params, baai_topic_model = cluster('baai_baseline_data2_semi_td_params.json', params, hf_model_name='BAAI/bge-base-en-v1.5')

# similarity_sbert_model, similarity_sbert_output_df, similarity_sbert_cluster_topics, similarity_sbert_params, similarity_sbert_topic_model = cluster('similarity_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-similarity")
# similarity_baai_model, similarity_baai_output_df, similarity_baai_cluster_topics, similarity_baai_params, similarity_baai_topic_model = cluster('best_similarity_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-similarity-best-baai")
# similarity_ai_car_sbert_model, similarity_ai_car_sbert_output_df, similarity_ai_car_sbert_cluster_topics, similarity_ai_car_sbert_params, similarity_ai_car_sbert_topic_model = cluster('similarity_ai_car_data2_semi_td_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-ai-car')
# similarity_distill_consistency_model, similarity_distill_consistency_output_df, similarity_distill_consistency_cluster_topics, similarity_distill_consistency_params, similarity_distill_consistency_topic_model = cluster('similarity_distill_consistency_data2_semi_td_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-distill-consistency')
# similarity_ai_car_distill_consistency_model, similarity_ai_car_distill_consistency_output_df, similarity_ai_car_distill_consistency_cluster_topics, similarity_ai_car_distill_consistency_params, similarity_ai_car_distill_consistency_topic_model = cluster('similarity_ai_car_distill_consistency_data2_semi_td_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-ai_car-distill-consistency')

# classification_model, classification_output_df, classification_cluster_topics, classification_params, classification_topic_model = cluster('classification_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification")
# classification_ai_car_sbert_model, classification_ai_car_sbert_output_df, classification_ai_car_sbert_cluster_topics, classification_ai_car_sbert_params, classification_ai_car_sbert_topic_model = cluster('classification_ai_car_sbert_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification-ai_car-sbert")
# combined_ai_car_class_consulting_sim_model, combined_ai_car_class_consulting_sim_output_df, combined_ai_car_class_consulting_sim_cluster_topics, combined_ai_car_class_consulting_sim_params, combined_ai_car_class_consulting_sim_topic_model = cluster('combined_ai_car_class_consulting_sim_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-class-consulting-sim")
# combined_consulting_sim_ai_car_class_model, combined_consulting_sim_ai_car_class_output_df, combined_consulting_sim_ai_car_class_cluster_topics, combined_consulting_sim_ai_car_class_params, combined_consulting_sim_ai_car_class_topic_model = cluster('combined_consulting_sim_ai_car_class_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-consulting-sim-ai_car-class")
# combined_consulting_sim_and_ai_car_class_model, combined_consulting_sim_and_ai_car_class_output_df, combined_consulting_sim_and_ai_car_class_cluster_topics, combined_consulting_sim_and_ai_car_class_params, combined_consulting_sim_and_ai_car_class_topic_model = cluster('combined_consulting_sim_and_ai_car_class_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-consulting-sim-and-ai_car-class")
# combined_ai_car_sim_and_class_model, combined_ai_car_sim_and_class_output_df, combined_ai_car_sim_and_class_cluster_topics, combined_ai_car_sim_and_class_params, combined_ai_car_sim_and_class_topic_model = cluster('combined_ai_car_sim_and_class_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-sim-and-class")
# combined_ai_car_sim_class_model, combined_ai_car_sim_class_output_df, combined_ai_car_sim_class_cluster_topics, combined_ai_car_sim_class_params, combined_ai_car_sim_class_topic_model = cluster('combined_ai_car_sim_class_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-sim-class")
# combined_ai_car_class_sim_model, combined_ai_car_class_sim_output_df, combined_ai_car_class_sim_cluster_topics, combined_ai_car_class_sim_params, combined_ai_car_class_sim_topic_model = cluster('combined_ai_car_class_sim_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-class-sim")
# combined_consulting_model, combined_consulting_output_df, combined_consulting_cluster_topics, combined_consulting_params, combined_consulting_topic_model = cluster('combined_consulting_data2_semi_td_params.json', params, hf_model_name="brjezierski/combined-consulting")

ex3_classification_ai_car_data_aug_model, ex3_classification_ai_car_data_aug_output_df, ex3_classification_ai_car_data_aug_cluster_topics, ex3_classification_ai_car_data_aug_params, ex3_classification_ai_car_data_aug_topic_model = cluster('classification_ai_car_data_aug_data2_semi_td_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification-ai_car-data_aug")


Input file ../glanos-data/clustering/input/classification_ai_car_data_aug_data2_semi_td_params.json
Supervised
Topic modelling method: BERTopic
Fitting the BERT model on 1278 snippets


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

2023-10-27 18:24:28,329 - BERTopic - Transformed documents to Embeddings
2023-10-27 18:24:30,865 - BERTopic - Reduced dimensionality
2023-10-27 18:24:30,881 - BERTopic - Clustered reduced embeddings


BERT model labelling 3940 snippets
inference_snippets 3940


Batches:   0%|          | 0/124 [00:00<?, ?it/s]

2023-10-27 18:25:17,313 - BERTopic - Reduced dimensionality
2023-10-27 18:25:17,366 - BERTopic - Predicted clusters


Elapsed total time for topic modelling: 67.36825299263 seconds
sbert results
Label count: 7
Loss: 0.006852791878172589
dataset_file_path ../glanos-data/clustering/big_consulting_export_data2_no_duplicates.tsv
Overlap count of length 1: 5
Overlap count of length 2: 2
Overlap count of length 3: 0
Overlap count of length 4 or more: 0
Overlap loss:  0.46
Diversity loss:  1.13
Number of clusters: 7
Avg cluster size: 544.5714285714286
Median cluster size: 169.0 , standard deviation: 706.8539669636473
Median top company occurence: 28.50678733031674
Cost (outliers): 0.69%



In [22]:
classification_ai_car_data_aug_cluster_topics

[['ibm',
  'ai',
  'consulting',
  'technology',
  'infosys',
  'services',
  'announced',
  'cloud',
  'wipro',
  'business'],
 ['mr',
  'ernst',
  'ernst young',
  'young',
  'prior',
  'partner',
  'llp',
  'years',
  'young llp',
  'joining'],
 ['pwc',
  'contract',
  'tax',
  'million',
  'secured',
  'contracts',
  'federal',
  'based',
  'secures',
  'department'],
 ['accenture',
  'domus',
  'sustainability',
  'green domus',
  'acquisition',
  'green',
  'acquire',
  'based',
  'digital',
  'announced'],
 ['fti',
  'appointed',
  'managing director',
  'director',
  'managing',
  'fti consulting',
  'tax',
  'jobs',
  'pwc',
  'joins'],
 ['patent',
  'application',
  'filed',
  'tata consultancy',
  'tata',
  'consultancy services',
  'consultancy',
  'services',
  'patents',
  'method'],
 ['microsoft',
  'salesforce',
  'leave',
  'employees',
  'ibm',
  'jobs',
  '000',
  'pwc',
  'reportedly',
  'layoffs']]

In [ ]:
data2_input_file = '../glanos-data/clustering/big_consulting_export_replace_no_duplicates.tsv'
print_results(input_file=data2_input_file)

# Experiment 4

In [15]:
# on replace_no_tags dataset
# overlap loss

data2_input_file = '../glanos-data/clustering/big_consulting_export_replace.tsv'
params['BAYESIAN_SEARCH_METRIC'] = 'topic_overlap+chat_intents'
params['MAX_EVALS'] = 15

sbert_model, sbert_output_df, sbert_cluster_topics, sbert_params, sbert_topic_model = cluster('baseline_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name='sentence-transformers/all-MiniLM-L12-v2')
baai_model, baai_output_df, baai_cluster_topics, baai_params, baai_topic_model = cluster('baai_baseline_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name='BAAI/bge-base-en-v1.5')

similarity_sbert_model, similarity_sbert_output_df, similarity_sbert_cluster_topics, similarity_sbert_params, similarity_sbert_topic_model = cluster('similarity_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-similarity")
similarity_baai_model, similarity_baai_output_df, similarity_baai_cluster_topics, similarity_baai_params, similarity_baai_topic_model = cluster('best_similarity_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-similarity-best-baai")
similarity_ai_car_sbert_model, similarity_ai_car_sbert_output_df, similarity_ai_car_sbert_cluster_topics, similarity_ai_car_sbert_params, similarity_ai_car_sbert_topic_model = cluster('similarity_ai_car_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-ai-car')
similarity_distill_consistency_model, similarity_distill_consistency_output_df, similarity_distill_consistency_cluster_topics, similarity_distill_consistency_params, similarity_distill_consistency_topic_model = cluster('similarity_distill_consistency_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-distill-consistency')
similarity_ai_car_distill_consistency_model, similarity_ai_car_distill_consistency_output_df, similarity_ai_car_distill_consistency_cluster_topics, similarity_ai_car_distill_consistency_params, similarity_ai_car_distill_consistency_topic_model = cluster('similarity_ai_car_distill_consistency_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name='brjezierski/sentence-embeddings-similarity-ai_car-distill-consistency')


classification_model, classification_output_df, classification_cluster_topics, classification_params, classification_topic_model = cluster('classification_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification")
classification_ai_car_sbert_model, classification_ai_car_sbert_output_df, classification_ai_car_sbert_cluster_topics, classification_ai_car_sbert_params, classification_ai_car_sbert_topic_model = cluster('classification_ai_car_sbert_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification-ai_car-sbert")
combined_ai_car_class_consulting_sim_model, combined_ai_car_class_consulting_sim_output_df, combined_ai_car_class_consulting_sim_cluster_topics, combined_ai_car_class_consulting_sim_params, combined_ai_car_class_consulting_sim_topic_model = cluster('combined_ai_car_class_consulting_sim_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-class-consulting-sim")
combined_consulting_sim_ai_car_class_model, combined_consulting_sim_ai_car_class_output_df, combined_consulting_sim_ai_car_class_cluster_topics, combined_consulting_sim_ai_car_class_params, combined_consulting_sim_ai_car_class_topic_model = cluster('combined_consulting_sim_ai_car_class_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-consulting-sim-ai_car-class")
combined_consulting_sim_and_ai_car_class_model, combined_consulting_sim_and_ai_car_class_output_df, combined_consulting_sim_and_ai_car_class_cluster_topics, combined_consulting_sim_and_ai_car_class_params, combined_consulting_sim_and_ai_car_class_topic_model = cluster('combined_consulting_sim_and_ai_car_class_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-consulting-sim-and-ai_car-class")
combined_ai_car_sim_and_class_model, combined_ai_car_sim_and_class_output_df, combined_ai_car_sim_and_class_cluster_topics, combined_ai_car_sim_and_class_params, combined_ai_car_sim_and_class_topic_model = cluster('combined_ai_car_sim_and_class_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-sim-and-class")
combined_ai_car_sim_class_model, combined_ai_car_sim_class_output_df, combined_ai_car_sim_class_cluster_topics, combined_ai_car_sim_class_params, combined_ai_car_sim_class_topic_model = cluster('combined_ai_car_sim_class_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-sim-class")
combined_ai_car_class_sim_model, combined_ai_car_class_sim_output_df, combined_ai_car_class_sim_cluster_topics, combined_ai_car_class_sim_params, combined_ai_car_class_sim_topic_model = cluster('combined_ai_car_class_sim_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-combined-ai_car-class-sim")
combined_consulting_model, combined_consulting_output_df, combined_consulting_cluster_topics, combined_consulting_params, combined_consulting_topic_model = cluster('combined_consulting_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/combined-consulting")

classification_ai_car_data_aug_model, classification_ai_car_data_aug_output_df, classification_ai_car_data_aug_cluster_topics, classification_ai_car_data_aug_params, classification_ai_car_data_aug_topic_model = cluster('classification_ai_car_data_aug_data2_semi_replace_no_tags_to_ci_params.json', params, hf_model_name="brjezierski/sentence-embeddings-classification-ai_car-data_aug")

Input file ../glanos-data/clustering/input/baseline_data2_semi_replace_no_tags_to_ci_params.json
Supervised
Topic modelling method: BERTopic
Fitting the BERT model on 1278 snippets


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

2023-10-26 23:36:28,364 - BERTopic - Transformed documents to Embeddings
2023-10-26 23:36:30,976 - BERTopic - Reduced dimensionality
2023-10-26 23:36:30,995 - BERTopic - Clustered reduced embeddings


BERT model labelling 3940 snippets
inference_snippets 3940


Batches:   0%|          | 0/124 [00:00<?, ?it/s]

...

2023-10-26 23:53:24,368 - BERTopic - Transformed documents to Embeddings
2023-10-26 23:53:26,675 - BERTopic - Reduced dimensionality
2023-10-26 23:53:26,687 - BERTopic - Clustered reduced embeddings


BERT model labelling 3940 snippets
inference_snippets 3940


Batches:   0%|          | 0/124 [00:00<?, ?it/s]

2023-10-26 23:53:56,804 - BERTopic - Reduced dimensionality
2023-10-26 23:53:56,849 - BERTopic - Predicted clusters


Elapsed total time for topic modelling: 49.1462767124176 seconds
sbert results
Label count: 6
Loss: 0.0025380710659898475
dataset_file_path ../glanos-data/clustering/big_consulting_export_replace_no_duplicates.tsv
Overlap count of length 1: 0
Overlap count of length 2: 1
Overlap count of length 3: 0
Overlap count of length 4 or more: 0
Overlap loss:  0.17
Diversity loss:  1.03
Number of clusters: 6
Avg cluster size: 640.1666666666666
Median cluster size: 583.0 , standard deviation: 549.8100934767284
Median top company occurence: 31.967871485943775
Cost (outliers): 0.25%



In [ ]:
print_results(input_file=data2_input_file)